In [11]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pickle as pkl
import time
import random
import re
import json

In [2]:
# Define the URL of the webpage with the information
# url = "https://www.transfermarkt.ch/bsc-young-boys/alumni/verein/452/ajax/yw1"
url = "https://www.transfermarkt.ch/bsc-young-boys/alumni/verein/452/buchstabe//land_id/0/position/alle/detailposition/alle/plus/1"

In [3]:
# Send a GET request and parse the HTML content
# headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
# }
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8", 
    "Accept-Encoding": "gzip, deflate, br", 
    "Accept-Language": "de,en-US;q=0.7,en;q=0.3",  
    "Sec-Fetch-Dest": "document", 
    "Sec-Fetch-Mode": "navigate", 
    "Sec-Fetch-Site": "cross-site", 
    "Sec-Fetch-User": "?1", 
    "Upgrade-Insecure-Requests": "1", 
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/112.0"
}

In [4]:
# List for row collection
player_data = []
soup_old = None
# Extract the player information from the table
for p in range(60): #there are about 50 pages for now but the exact number will change, so try to load until fail
    print(p, 'waiting')
    time.sleep(random.uniform(1, 2))
    if p==0: page = ""
    else: page = f"/page/{p+1}"
    start_time = time.time()
    try:
        response = requests.get(url+page, headers=headers)
        # process the response here
    except requests.exceptions.RequestException:
        print(f"Error at {url+page}")
        break
    elapsed_time = time.time() - start_time
    print(f"Request of {url+page} took {elapsed_time} seconds")

    soup = BeautifulSoup(response.content, "html.parser")
    if soup_old == soup:
        print("End of pages reached") #if the same page is loaded twice, we reached the end
        break
    soup_old = soup
    

    # Find the table with the player data
    player_table = soup.find("table", class_="items")
    
    # for each row get the relevant data. Only every 4. row holds the full player data
    for r,row in enumerate(player_table.find_all("tr")[1::7]):
        player_info = {}
        columns = row.find_all("td")

        player_info["player_image_link"] = columns[1].find("img")["data-src"]
        player_info["player_name"] = columns[2].find("a")["title"]
        player_info["player_profile_link"] = "https://www.transfermarkt.ch"+columns[2].find("a")["href"]
        player_info["player_position"] = columns[3].text.strip()
        player_info["player_birthday"] = columns[8].text.strip()
        player_info["player_nationality"] = columns[10].find("img")["alt"]
        player_info["player_left"] = columns[12].text.strip()
        player_info["player_team"] = columns[14].find("a").text.strip()
        
        player_data.append(player_info)

player_data = pd.DataFrame(player_data)
player_data = player_data.drop_duplicates().reset_index(drop=True)
# Save the data as a pickle file
with open("Data/player_data.pkl", "wb") as f:
    pkl.dump(player_data, f)


0 waiting
Request of https://www.transfermarkt.ch/bsc-young-boys/alumni/verein/452/buchstabe//land_id/0/position/alle/detailposition/alle/plus/1 took 0.23972249031066895 seconds
1 waiting
Request of https://www.transfermarkt.ch/bsc-young-boys/alumni/verein/452/buchstabe//land_id/0/position/alle/detailposition/alle/plus/1/page/2 took 16.166462182998657 seconds
2 waiting
Request of https://www.transfermarkt.ch/bsc-young-boys/alumni/verein/452/buchstabe//land_id/0/position/alle/detailposition/alle/plus/1/page/3 took 14.826876878738403 seconds
3 waiting
Request of https://www.transfermarkt.ch/bsc-young-boys/alumni/verein/452/buchstabe//land_id/0/position/alle/detailposition/alle/plus/1/page/4 took 29.047762870788574 seconds
4 waiting
Request of https://www.transfermarkt.ch/bsc-young-boys/alumni/verein/452/buchstabe//land_id/0/position/alle/detailposition/alle/plus/1/page/5 took 19.785176992416382 seconds
5 waiting
Request of https://www.transfermarkt.ch/bsc-young-boys/alumni/verein/452/buc

In [14]:
# Load the data from the pickle file
with open("../Data/player_data.pkl", "rb") as f:
    player_data = pkl.load(f)

In [4]:
player_data

,player_image_link,player_name,player_profile_link,player_position,player_birthday,player_nationality,player_left,player_team
0,https://img.a.transfermarkt.technology/portrai...,Denis Zakaria,https://www.transfermarkt.ch/denis-zakaria/pro...,Defensives Mittelfeld,20.11.1996,Schweiz,01.07.2017,1.
1,https://img.a.transfermarkt.technology/portrai...,Djibril Sow,https://www.transfermarkt.ch/djibril-sow/profi...,Zentrales Mittelfeld,06.02.1997,Schweiz,01.07.2019,1.
2,https://img.a.transfermarkt.technology/portrai...,Seydou Doumbia,https://www.transfermarkt.ch/seydou-doumbia/pr...,Mittelstürmer,31.12.1987,Elfenbeinküste,01.07.2010,1.
3,https://img.a.transfermarkt.technology/portrai...,Josef Martínez,https://www.transfermarkt.ch/josef-martinez/pr...,Mittelstürmer,19.05.1993,Venezuela,01.07.2014,1.
4,https://img.a.transfermarkt.technology/portrai...,Miralem Sulejmani,https://www.transfermarkt.ch/miralem-sulejmani...,Linkes Mittelfeld,05.12.1988,Serbien,01.07.2022,1.
...,...,...,...,...,...,...,...,...
1219,https://img.a.transfermarkt.technology/portrai...,? Scheidegger,https://www.transfermarkt.ch/-scheidegger/prof...,Mittelfeld,-,Schweiz,-,1.
1220,https://img.a.transfermarkt.technology/portrai...,Martin Bircher,https://www.transfermarkt.ch/martin-bircher/pr...,Mittelfeld,-,Schweiz,-,1.
1221,https://img.a.transfermarkt.technology/portrai...,Etienne Moser,https://www.transfermarkt.ch/etienne-moser/pro...,Torwart,02.07.1995,Schweiz,-,Jugend
1222,https://img.a.transfermarkt.technology/portrai...,Matteo Blumenthal,https://www.transfermarkt.ch/matteo-blumenthal...,Abwehr,13.03.1996,Schweiz,-,Jugend


#### Reduce Players

In [29]:
# select only players that played in the first team
selection = player_data[player_data["player_team"]=="1."]
# extract the date from player_left
# first sort out entries with "-"
selection = selection[selection["player_left"]>"-"]
# then replace all entries that contain vsl. bis with None (not left yet)
selection["player_left"] = selection["player_left"].apply(lambda x: None if "vsl. bis" in x else x)
# then convert dates to datetime
selection["player_left"] = pd.to_datetime(selection["player_left"], format="%d.%m.%Y")
# then select only players that left after 2000 or are still in the team
selection = selection[(selection["player_left"]>pd.to_datetime("2022-01-01")) | (selection["player_left"].isna())]
# convert dates back to string
selection["player_left"] = selection["player_left"].apply(lambda x: x.strftime("%d.%m.%Y") if not pd.isna(x) else None)
# reset index
selection = selection.reset_index(drop=True)

In [30]:
selection.head()

,player_image_link,player_name,player_profile_link,player_position,player_birthday,player_nationality,player_left,player_team
0,https://img.a.transfermarkt.technology/portrai...,Miralem Sulejmani,https://www.transfermarkt.ch/miralem-sulejmani...,Linkes Mittelfeld,05.12.1988,Serbien,01.07.2022,1.
1,https://img.a.transfermarkt.technology/portrai...,Jordan,https://www.transfermarkt.ch/jordan/profil/spi...,Mittelstürmer,26.04.1996,Vereinigte Staaten,01.07.2022,1.
2,https://img.a.transfermarkt.technology/portrai...,Fabian Rieder,https://www.transfermarkt.ch/fabian-rieder/pro...,Offensives Mittelfeld,16.02.2002,Schweiz,None,1.
3,https://img.a.transfermarkt.technology/portrai...,Jean-Pierre Nsame,https://www.transfermarkt.ch/jean-pierre-nsame...,Mittelstürmer,01.05.1993,Kamerun,None,1.
4,https://img.a.transfermarkt.technology/portrai...,Edimilson Fernandes,https://www.transfermarkt.ch/edimilson-fernand...,Zentrales Mittelfeld,15.04.1996,Schweiz,30.06.2022,1.


### Request the transfer history of players

In [31]:
# Write a function that requests the transfer history of a player based on the player profile link using the requests library
def requestTransferHistory(player_profile_link,headers):
    player_transfer = []
    start_time = time.time()
    try:
        response = requests.get(player_profile_link, headers=headers)
    except requests.exceptions.RequestException:
        print(f"Error at {player_profile_link}")
        return None
    elapsed_time = time.time() - start_time
    # print(f"Request of {player_profile_link} took {elapsed_time} seconds")  
    soup = BeautifulSoup(response.content, "html.parser")
    # Find the table with the player data
    transfer_table = soup.find_all("div", class_="tm-player-transfer-history-grid")
    # exclude rows with class="tm-player-transfer-history-grid--heading
    transfer_table = [row for row in transfer_table if "tm-player-transfer-history-grid--heading" not in row.get('class')]
    for r,row in enumerate(transfer_table):
        transfer_info = {}
        columns = row.find_all("div")
        transfer_info["season"] = columns[0].text.strip()
        transfer_info["transfer_date"] = columns[1].text.strip()
        transfer_info["team_left_name"] = columns[2].text.strip()
        transfer_info["team_left_logo"] = columns[2].find("img", class_="tm-player-transfer-history-grid__club-logo")["data-srcset"]
        transfer_info["team_left_logo"] = re.search("(?P<url>https?://[^\s]+)", transfer_info["team_left_logo"]).group("url") #extract the url from the srcset
        try: transfer_info["team_left_flag"] = columns[2].find("img", class_="tm-player-transfer-history-grid__flag")["data-src"]
        except: transfer_info["team_left_flag"] = None
        transfer_info["team_joined_name"] = columns[3].text.strip()
        transfer_info["team_joined_logo"] = columns[3].find("img", class_="tm-player-transfer-history-grid__club-logo")["data-srcset"]
        transfer_info["team_joined_logo"] = re.search("(?P<url>https?://[^\s]+)", transfer_info["team_joined_logo"]).group("url") #extract the url from the srcset
        try: transfer_info["team_joined_flag"] = columns[3].find("img", class_="tm-player-transfer-history-grid__flag")["data-src"]
        except: transfer_info["team_joined_flag"] = None
        transfer_info["market_value"] = columns[4].text.strip()
        transfer_info["transfer_fee"] = columns[5].text.strip()
        player_transfer.append(transfer_info)
    return player_transfer


In [32]:
selection["player_transfer_history"] = None
for i,player in selection.iterrows():
    player_transfer = requestTransferHistory(player.player_profile_link,headers)
    selection["player_transfer_history"].loc[i]=player_transfer
    time.sleep(random.uniform(1, 2))

In [33]:
out = selection.to_dict(orient="records")

In [34]:
with open(f"../Data/transfers.json", "w") as file:
    json.dump(out, file)